In [10]:
import json
import random
from collections import defaultdict
import math

class CobwebNode():
	def __init__(self):
		self.instance_count = 0
		self.attribute_value = defaultdict(lambda: defaultdict(int))
		self.label = None
		self.children = []
		self.class_distribution = {0:0, 1:0}

	def increment(self, attribute, value):
		self.attribute_value[attribute][value] += 1
	
	def decrement(self, attribute, value):
		self.attribute_value[attribute][value] -= 1


class LearningSystem(object):
	"""
	Set up a learning system class, i.e., the approach you will apply.
	Feel free to create any functions under the class.

	Two functions are REQUIRED:
	- LearningSystem.fit(train_set, train_labels)
	- LearningSystem.predict(instance)
	"""

	def __init__(self):
		"""
		You can have any other parameters here.
		"""
		self.root = CobwebNode()
	
	def concept_utility(self, node):
		# print("concept_utility")
		cu = 0
		for attribute, val_dict in node.attribute_value.items():
			for value, count in val_dict.items():
				prob = count / node.instance_count
				cu += prob * (prob - 1/node.instance_count)
		return cu
	
	def hypothetical_cu(self, node, instance, label, op):
		# print("hypothetical_cu - ", op)
		current_cu = self.calculate_cu(node)

		if op == "add_to_best":
			best_child = self.best_child_for_instance(node, instance)
			if best_child:
				self.insert_tmp(best_child, instance, label)
				new_cu = self.calculate_cu(node)
				self.remove_tmp(best_child, instance, label)
			
			else:
				new_cu = current_cu
			
		if op == "new_child":
			# add and remove new child temporarily
			new_child = CobwebNode()
			original_children = node.children.copy()
			self.insert_tmp(new_child, instance, label)
			node.children.append(new_child)
			new_cu = self.calculate_cu(node)
			self.remove_tmp(new_child, instance, label)
			node.children = original_children
		
		if op == "merge":
			original_children = node.children.copy()
			self.merge_most_similar(node)
			new_cu = self.calculate_cu(node)
			node.children = original_children
		
		return new_cu - current_cu

	# def best_child_for_instance(self, node, instance):
	# 	max_delta_cu = float('-inf')
	# 	best_child = None

	# 	for child in node.children:
	# 		delta_cu = self.hypothetical_cu(node, instance, child, "add_to_best")
	# 		if delta_cu > max_delta_cu:
	# 			max_delta_cu = delta_cu
	# 			best_child = child
		
	# 	return best_child

	# def best_child_for_instance(self, node, instance):
	# 	# 개선 가능성 있음. 
	# 	max_match = 0
	# 	best_child = None

	# 	for child in node.children:
	# 		matching_attributes = sum(1 for attr, value in instance.items() if child.attribute_value.get(attr, {}).get(value, 0) > 0)
	# 		if matching_attributes > max_match:
	# 			max_match = matching_attributes
	# 			best_child = child
		
	# 	return best_child

	def best_child_for_instance(self, node, instance):
		best_score = float('-inf')
		best_child = None

		for child in node.children:
			score = 0
			for attr, value in instance.items():
				score += child.attribute_value.get(attr, {}).get(value, 0) / child.instance_count
			if score > best_score:
				best_score = score
				best_child = child
			
		return best_child
			
	
	def insert_tmp(self, node, instance, label):
		node.instance_count +=1

		if label in node.class_distribution:
			node.class_distribution[label] += 1
		
		for attribute, value in instance.items():
			if value not in node.attribute_value[attribute]:
				node.attribute_value[attribute][value] = 0
			node.attribute_value[attribute][value] += 1

	def remove_tmp(self, node, instance, label):
		node.instance_count -= 1
		if label in node.class_distribution:
			node.class_distribution[label] -= 1
		
		for attribute, value in instance.items():
			if attribute in node.attribute_value and value in node.attribute_value[attribute]:
				node.attribute_value[attribute][value] -= 1
				if node.attribute_value[attribute][value] == 0:
					del node.attribute_value[attribute][value]
	
	def calculate_cu(self, node):
		if not node.children:
			return 0
		
		total_instances = node.instance_count
		total_cu = 0

		for child in node.children:
			pi = child.instance_count / total_instances
			ei = self.calculate_entropy(child)
			total_cu += pi * ei
		
		return total_cu
	
	def calculate_entropy(self, node):
		total = node.instance_count
		if total == 0:
			return 0
		probs = [node.class_distribution[0]/total, node.class_distribution[1]/total]
		entropy = sum([-p * math.log(p, 2) if p > 0 else 0 for p in probs])
		return entropy

	# def merge_most_similar(self, node):
	# 	best_pair = None
	# 	best_sim = -1

	# 	for i in range(len(node.children)):
	# 		for j in range(i+1, len(node.children)):
	# 			sim = self.calculate_sim(node.children[i], node.children[j])
	# 			if sim > best_sim:
	# 				best_sim = sim
	# 				best_pair = (node.children[i], node.children[j])

	# 	if best_pair:
	# 		merged_node = self.merge_nodes(best_pair[0], best_pair[1])
	# 		node.children.remove(best_pair[0])
	# 		node.children.remove(best_pair[1])
	# 		node.children.append(merged_node)

	def merge_most_similar(self, node):
		best_score = float('-inf')
		best_pair = None

		for i in range(len(node.children)):
			for j in range(i+1, len(node.children)):
				sim_score = self.calculate_merge_score(node.children[i], node.children[j])
				if sim_score > best_score:
					best_pair = sim_score
					best_pair = (node.children[i], node.children[j])

		if best_pair:
			merged_node = self.merge_nodes(best_pair[0], best_pair[1])
			node.children.remove(best_pair[0])
			node.children.remove(best_pair[1])
			node.children.append(merged_node)

	def calculate_merge_score(self, node1, node2):
		total_count = node1.instance_count + node2.instance_count
		overlapping_attributes = set(node1.attribute_value.keys()) & set(node2.attribute_value.keys())
		score = 0
		for attr in overlapping_attributes:
			for value in set(node1.attribute_value[attr].keys()) | set(node2.attribute_value[attr].keys()):
				score += abs((node1.attribute_value[attr].get(value, 0) / node1.instance_count) - (node2.attribute_value[attr].get(value, 0) / node2.instance_count))	

		return score	



	def calculate_sim(self, node1, node2):
		return sum([abs(node1.class_distribution[k] - node2.class_distribution[k]) for k in [0,1]])
	
	def merge_nodes(self, node1, node2):
		merged = CobwebNode()
		merged.instance_count = node1.instance_count + node2.instance_count

		for k in [0,1]:
			merged.class_distribution[k] = node1.class_distribution[k] + node2.class_distribution[k]

		for attribute, val_dict in node1.attribute_value.items():
			for value, count in val_dict.items():
				merged.attribute_value[(attribute, value)] = count
		
		for attribute in set(node1.attribute_value.keys()) | set(node2.attribute_value.keys()):
			for value in set(node1.attribute_value[attribute].keys()) | set(node2.attribute_value[attribute].keys()):
				merged.attribute_value[attribute][value] = node1.attribute_value[attribute].get(value, 0) + node2.attribute_value[attribute].get(value, 0)
		
		merged.children.extend(node1.children)
		merged.children.extend(node2.children)

		return merged

	def insert(self, node, instance, label):
		node.instance_count += 1
		node.class_distribution[label] += 1
		for attribute, value in instance.items():
			node.increment(attribute, value)

		operations = ["add_to_best", "new_child", "merge"]
		best_op = None
		best_cu = -1

		for op in operations:
			cu = self.hypothetical_cu(node, instance, label, op)
			# print("CU: ", cu, " FB: ", best_cu, " OP: ", op)
			if cu > best_cu:
				best_cu = cu
				best_op = op
		
		if best_op == "add_to_best":
			best_child = self.best_child_for_instance(node, instance)
			if best_child:
				self.insert(best_child, instance, label)
				# print("INSERT - ", best_op)
			else:
				best_op = "new_child"
		
		if best_op == "new_child":
			new_child = CobwebNode()
			for attribute, value in instance.items():
				new_child.increment(attribute, value)
			new_child.instance_count = 1
			new_child.class_distribution[label] = 1
			node.children.append(new_child)
			# print("INSERT - ", best_op)
		
		if best_op == "merge":
			self.merge_most_similar(node)
			# print("INSERT - ", best_op)
		
		node.label = max(node.class_distribution, key=lambda k: node.class_distribution[k])
		return


	def fit(self, train_set, train_labels):
		"""
		Train the system with this function.
		Do NOT set any input variable other than train_set and train_labels for this function.
		========
		train_data: training data without labels. It should be a list of dictionaries.
		train_labels: A list of labels of training data. Each label has the same index as its data in train_data.
					  All labels are binary (0, 1).
		"""
		for i, instance in enumerate(train_set):
			self.insert(self.root, instance, train_labels[i])


	def predict(self, instance):
		"""
		Predict the label of an instance (a single dictionary) given.
		"""
		return self._predict_recursive(self.root, instance)
	
	# def _predict_recursive(self, node, instance):
	# 	if not node.children:
	# 		return node.label
		
	# 	best_child = None
	# 	best_cu = -1
	# 	for child in node.children:
	# 		cu = self.concept_utility(child)
	# 		if cu > best_cu:
	# 			best_cu = cu
	# 			best_child = child
	# 	return self._predict_recursive(best_child, instance)

	def _predict_recursive(self, node, instance):
		if not node.children:
			return node.label or max(node.class_distribution, key=lambda k: node.class_distribution[k])
		
		best_child = self.best_child_for_instance(node, instance)
		if best_child:
			return self._predict_recursive(best_child, instance)
		return max(node.class_distribution, key=lambda k: node.class_distribution[k])


# Then you need to make the separate define_model() function,
# so you can have any initialized parameter defined.
# Autograder will define the model object based on your define_model() function.
def define_model():
	return LearningSystem()


"""
The following codes are for testing with sample data only.
REMEMBER TO REMOVE THEM BEFORE SUBMISSION.
=============================================================================
In this homework, a sample collection of shuffled data is provided.
You might try with the data provided like in the following before submission:
"""

def test(model, test_set_data, test_labels):
	correct = 0
	for i in range(len(test_set_data)):
		pred = model.predict(test_set_data[i])
		if pred == test_labels[i]:
			correct += 1
	accuracy = correct / len(test_set_data)
	return accuracy

In [11]:
sample_set_data = json.load(open('hw3-sample_set_data.json'))
sample_set_labels = json.load(open('hw3-sample_set_labels.json'))

train_set = sample_set_data[:-500]
train_labels = sample_set_labels[:-500]
test_set = sample_set_data[-500:]
test_labels = sample_set_labels[-500:]

model = define_model()
model.fit(train_set, train_labels)
print("Accuracy:", test(model, test_set, test_labels))

AttributeError: 'LearningSystem' object has no attribute 'merge_most_similar'